### Goal
Run flows using sdk and allowing for external configuration of RAG parameters. 

### Check Connections

In [1]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

acs_connection (CognitiveSearch)
aoai_connection (AzureOpenAI)
cosmodb_connection (Custom)
postgres_connection (Custom)


### Test the flow

In [2]:
output = pf.flows.test(
    "../../rag-cosmos-mongo/flow.dag.yaml",
    inputs={
        "chat_history": [],
        "query": "What is the growth rate of Azure ML revenue in FY23Q1?",
    },
)

print(output)

[2024-06-21 09:37:01,171][promptflow.core._connection][WARNING] - Please use connection.secrets[key] to access secrets.


2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Start to run 15 nodes with concurrency level 16.
2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Executing node ConfigLoader. node run id: ffcf9c38-9628-4d3f-a2d9-e65d02b51a22_ConfigLoader_0
2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Executing node CheckExistingContext. node run id: ffcf9c38-9628-4d3f-a2d9-e65d02b51a22_CheckExistingContext_0
2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Executing node FormatConversation. node run id: ffcf9c38-9628-4d3f-a2d9-e65d02b51a22_FormatConversation_0
2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Executing node QueryRewriter. node run id: ffcf9c38-9628-4d3f-a2d9-e65d02b51a22_QueryRewriter_0
2024-06-21 09:37:03 -0700   25192 execution.flow     INFO     Node CheckExistingContext completes.
2024-06-21 09:37:03 -0700   251

### Run the flow with the benchmark data file

In [3]:
flow_path = "../../rag-cosmos-mongo/flow.dag.yaml"
data_path = "../../datasets/evalset.csv"

column_mapping = {
    "chat_history": "${data.chat_history}",
    "query": "${data.query}",
}

run_mongo_topk3 = pf.run(flow=flow_path, data=data_path, column_mapping=column_mapping)
pf.stream(run_mongo_topk3)
print(run_mongo_topk3)

[2024-06-21 09:37:11,202][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_0_20240621_093710_442103, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_0_20240621_093710_442103\logs.txt


2024-06-21 09:37:16 -0700   25192 execution.bulk     INFO     Current system's available memory is 36515.11328125MB, memory consumption of current process is 208.5234375MB, estimated available worker count is 36515.11328125/208.5234375 = 175
2024-06-21 09:37:16 -0700   25192 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 13, 'estimated_worker_count_based_on_memory_usage': 175}.
2024-06-21 09:37:21 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-4)-Process id(25400)-Line number(0) start execution.
2024-06-21 09:37:21 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-5)-Process id(22036)-Line number(2) start execution.
2024-06-21 09:37:21 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-3)-Process id(16936)-Line number(1) start execution.
2024-06-21 09:37:21 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(

In [4]:
pf.get_details(run_mongo_topk3)

,inputs.chat_history,inputs.query,inputs.line_number,outputs.reply,outputs.fetched_docs,outputs.log_params
0,[],What is the quarterly revenue of Microsoft Clo...,0,The Microsoft Cloud exceeded $25 billion in qu...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
1,[],How many Azure Arc customers does Microsoft cu...,1,The number of Azure Arc customers in FY23Q1 wa...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
2,[],Which companies run SAP on Azure in FY23Q1?,2,"""Companies like Chobani, Munich RE, Sodexo, Vo...","{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
3,[],What is the annual recurring revenue of GitHub...,3,The requested information is not available in ...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
4,[],How many monthly active users does Power BI ha...,4,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
5,[],What is the quarterly revenue of Microsoft Clo...,5,"According to Satya Nadella, the Microsoft Clou...","{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
6,[],How many Azure Arc customers does Microsoft cu...,6,"More than 12,000 Azure Arc customers, double t...","{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
7,[],What are some examples of companies using Azur...,7,The Azure OpenAI Service was made broadly avai...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
8,[],How many developers are using GitHub Copilot a...,8,The retrieved documents do not provide any inf...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
9,[],What is the growth rate of Azure Machine Learn...,9,The growth rate of Azure Machine Learning reve...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."


In [5]:
ddf = pf.get_details(run_mongo_topk3)
#ddf.to_csv("checkoutput.csv")

## Evaluate 

In [7]:
eval_flow_path = "../../evaluator/eval_aistudio_score/"
data_path = "../../datasets/evalset.csv"

eval_mongo_run_topk3 = pf.run(
    flow=eval_flow_path,
    run=run_mongo_topk3,
    data=data_path,
    column_mapping={
        "question": "${data.query}",
        "answer": "${data.answer}",
        "reply": "${run.outputs.reply}",
        "context": "${run.outputs.fetched_docs}",
        "log_params": "${run.outputs.log_params}"
    },
    display_name="eval_mongo_aistudio_score_topk3",
)
pf.stream(eval_mongo_run_topk3)
print(eval_mongo_run_topk3)

[2024-06-21 09:39:09,452][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_093909_167267, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_093909_167267\logs.txt


2024-06-21 09:39:13 -0700   25192 execution.bulk     INFO     Current system's available memory is 36415.6640625MB, memory consumption of current process is 223.1796875MB, estimated available worker count is 36415.6640625/223.1796875 = 163
2024-06-21 09:39:13 -0700   25192 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 13, 'estimated_worker_count_based_on_memory_usage': 163}.
2024-06-21 09:39:18 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-7)-Process id(29400)-Line number(0) start execution.
2024-06-21 09:39:18 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-8)-Process id(25360)-Line number(1) start execution.
2024-06-21 09:39:18 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-9)-Process id(17944)-Line number(2) start execution.
2024-06-21 09:39:18 -0700   25192 execution.bulk     INFO     Process name(SpawnProcess-10)-Process id(2

In [8]:
pf.get_details(eval_mongo_run_topk3)

,inputs.question,inputs.answer,inputs.reply,inputs.context,inputs.log_params,inputs.line_number,outputs.relevance,outputs.coherence,outputs.fluency,outputs.groundedness,outputs.gpt_similarity
0,What is the quarterly revenue of Microsoft Clo...,"$25 billion in quarterly revenue, up 24 percen...",The Microsoft Cloud exceeded $25 billion in qu...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",0,5,5,4,5,5
1,How many Azure Arc customers does Microsoft cu...,"More than 8,500 Arc customers, more than doubl...",The number of Azure Arc customers in FY23Q1 wa...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",1,5,5,4,5,5
2,Which companies run SAP on Azure in FY23Q1?,"Companies like Chobani, Munich RE, Sodexo, Vol...","""Companies like Chobani, Munich RE, Sodexo, Vo...","{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",2,5,5,5,5,5
3,What is the annual recurring revenue of GitHub...,GitHub is now at $1 billion in annual recurrin...,The requested information is not available in ...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",3,1,5,4,1,1
4,How many monthly active users does Power BI ha...,Power BI is the market leader in business inte...,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",4,1,5,4,1,1
5,What is the quarterly revenue of Microsoft Clo...,"$27 billion in quarterly revenue, up 22 percen...","According to Satya Nadella, the Microsoft Clou...","{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",5,5,5,4,5,5
6,How many Azure Arc customers does Microsoft cu...,"More than 12,000 Arc customers, double the num...","More than 12,000 Azure Arc customers, double t...","{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",6,5,5,4,5,5
7,What are some examples of companies using Azur...,"Over 200 customers, including KPMG and Al Jaze...",The Azure OpenAI Service was made broadly avai...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",7,5,5,5,5,1
8,How many developers are using GitHub Copilot a...,More than one million people have used GitHub ...,The retrieved documents do not provide any inf...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",8,1,5,4,3,1
9,What is the growth rate of Azure Machine Learn...,Azure ML revenue alone has increased more than...,The growth rate of Azure Machine Learning reve...,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",9,1,4,4,1,1


In [9]:
pf.get_metrics(eval_mongo_run_topk3)

{'relevance': 2.8461538461538463,
 'coherence': 4.3076923076923075,
 'fluency': 3.8461538461538463,
 'groundedness': 3.0,
 'gpt_similarity': 2.5384615384615383}

In [10]:
pf.visualize(eval_mongo_run_topk3)

The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-qtrli0tm.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


### Multiple runs 
Using config.yaml file

In [11]:
import yaml
import promptflow

pf = promptflow.PFClient()

with open("./runs_config.yaml", "r") as file:
    runs_config = yaml.safe_load(file)

# Access the contents of the config.yaml file
print(runs_config)

{'Run1': {'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 3}, 'Run2': {'maxTokens': 3500, 'searchType': 'filter_vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 3}, 'Run3': {'maxTokens': 3500, 'searchType': 'filter_vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 5}}


In [12]:
# function to update param_config systematically 
import os 
def update_param_config(flow_path, config):
    base_dir = os.path.dirname(flow_path)
    param_config_path = os.path.join(base_dir, 'param_config.yaml')

    with open(param_config_path, 'r') as file:
        param_dict = yaml.safe_load(file)

    param_dict.update(config)

    with open(param_config_path, 'w') as file:
        yaml.dump(param_dict, file)


In [19]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def run_dags(
    flow_path="../../rag-cosmos-mongo/flow.dag.yaml",
    data_path="../../datasets/evalset.csv",
    eval_flow_path="../../evaluator/eval_aistudio_score/",
    config=None,
):
    column_mapping = {
        "chat_history": "${data.chat_history}",
        "query": "${data.query}",
    }
    update_param_config(flow_path, config)

    copilot_run_from_config = pf.run(
        flow=flow_path,
        data=data_path,
        column_mapping=column_mapping,
        display_name="rag_mongo_run_from_config_" + run_time,
    )


    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
        flow=eval_flow_path,
        run=copilot_run_from_config,
        data=data_path,
        column_mapping={
            "question": "${data.query}",
            "answer": "${data.answer}",
            "reply": "${run.outputs.reply}",
            "context": "${run.outputs.fetched_docs}",
            "log_params": "${run.outputs.log_params}",
        },
        display_name="eval_run_mongo_aistudio_scores_from_config_" + run_time,
    )
    return copilot_run_from_config, eval_run_from_config

In [20]:
i = 0
run_logs = {}
for run in runs_config:
    i += 1
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(config=runs_config[run])
    run_logs[i] = eval_run_from_config

{'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 3}


[2024-06-21 09:50:54,639][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_0_20240621_095054_367240, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_0_20240621_095054_367240\logs.txt
[2024-06-21 09:52:11,064][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_095210_798494, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_095210_798494\logs.txt


{'maxTokens': 3500, 'searchType': 'filter_vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 3}


[2024-06-21 09:55:29,243][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_0_20240621_095528_946440, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_0_20240621_095528_946440\logs.txt
[2024-06-21 09:56:43,253][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_095642_991343, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_095642_991343\logs.txt


{'maxTokens': 3500, 'searchType': 'filter_vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 5}


[2024-06-21 10:00:00,810][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_0_20240621_100000_465774, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_0_20240621_100000_465774\logs.txt
[2024-06-21 10:01:17,615][promptflow._sdk._submitter.run_submitter][WARNING] - 1 out of 13 runs failed in batch run.
 Please check out C:/Users/alsavelv/.promptflow/.runs/rag_cosmos_mongo_variant_0_20240621_100000_465774 for more details.
[2024-06-21 10:01:18,396][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_100118_012522, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_100118_012522\logs.txt


In [21]:
for run_id in run_logs:
    print(run_id)
    pf.visualize(run_logs[run_id])

1
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-d5xbyuvr.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
2
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-xc7l_k1k.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
3
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-6e66g4jn.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


### Run different flow variants
Now, lets evaluate same flow but on a node variant.

In [22]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def run_dags(
    flow_path="../../rag-cosmos-mongo/flow.dag.yaml",
    data_path="../../datasets/evalset.csv",
    eval_flow_path="../../evaluator/eval_aistudio_score/",
    variant=None,
    config=None,
):
    column_mapping = {
        "chat_history": "${data.chat_history}",
        "query": "${data.query}",
    }
    update_param_config(flow_path, config)

    copilot_run_from_config = pf.run(
        flow=flow_path,
        data=data_path,
        column_mapping=column_mapping,
        variant =variant,
        display_name="rag_mongo_run__variant_" + variant + "_from_config_" + run_time,
    )

    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
        flow=eval_flow_path,
        run=copilot_run_from_config,
        data=data_path,
        column_mapping={
            "question": "${data.query}",
            "answer": "${data.answer}",
            "reply": "${run.outputs.reply}",
            "context": "${run.outputs.fetched_docs}",
            "log_params": "${run.outputs.log_params}",
        },
        display_name="eval_run_mongo_variant_"+ variant + "_aistudio_scores_from_config_" + run_time,
    )
    return copilot_run_from_config, eval_run_from_config

In [23]:
i = 0
run_logs = {}
for run in runs_config:
    i += 1
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(variant= "${DetermineReply.variant_1}", config=runs_config[run])
    run_logs[i] = eval_run_from_config

{'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 3}


[2024-06-21 10:09:19,683][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_1_20240621_100919_377206, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_1_20240621_100919_377206\logs.txt
[2024-06-21 10:12:59,162][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_101258_679615, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_101258_679615\logs.txt


{'maxTokens': 3500, 'searchType': 'filter_vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 3}


[2024-06-21 10:16:17,413][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_1_20240621_101617_104216, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_1_20240621_101617_104216\logs.txt
[2024-06-21 10:19:57,083][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_101956_733993, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_101956_733993\logs.txt


{'maxTokens': 3500, 'searchType': 'filter_vector', 'dbName': 'earning_calls', 'colName': 'transcript_filter_vector', 'topK': 5}


[2024-06-21 10:23:16,566][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_mongo_variant_1_20240621_102316_326616, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_mongo_variant_1_20240621_102316_326616\logs.txt
[2024-06-21 10:26:50,265][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240621_102649_987377, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240621_102649_987377\logs.txt


In [24]:
for run_id in run_logs:
    print(run_id)
    pf.visualize(run_logs[run_id])

1
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-lae7cp9q.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
2
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-pcmh_bfs.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
3
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-x2ngdz2i.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
